# Tutorial: Credible Intervals/Regions

Here we'll compare different conventions for defining "best values" and credible intervals (CI's) to summarize PDFs:
1. Mode and highest posterior density
2. Median and quantiles

We'll compute these quantities from *samples* of a PDF. In addition, you will compare to the mean and standard deviation, though note that in general, unless your PDF is a Gaussian, those intervals don't encompass the probability they claim to.

In [ ]:
exec(open('tbc.py').read()) # define TBC and TBC_above
import numpy as np
import scipy.stats as st
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
%matplotlib inline

Here are some concrete test cases to work with. We'll find 1D values/CI's for these, and then turn to the 2D case where `x=test1D_1` and `y=test1D_2`.

In [ ]:
test1D_1 = st.beta.rvs(1.5, 5.0, size=100000)
test1D_2 = np.concatenate((st.norm.rvs(loc=-2.0, size=60000), st.norm.rvs(loc=2.0, size=40000)))

We'll find intervals corresponding to "$1\sigma$" and "$2\sigma$" equivalent probability. Recall that these are $\approx$0.683 and 0.954, or more precisely:

In [ ]:
st.chi2.cdf([1.0, 4.0], df=1)

## 1D intervals

Let's start with the 1D best value and CI for `test_1D_1`.

In [ ]:
plt.hist(test1D_1, bins=100, density=True, histtype='step');
plt.xlabel('test1D_1');

First, we'll find the mode and highest posterior density CI's using the only package we know of that does the latter correctly.

In [ ]:
import incredible as cr

With this package, there are actually two steps. The first converts the samples into a histogram, optionally doing some smoothing. After that, there is a second call to find the mode and CI's. This is so that the second function can also be used in cases where we have, e.g., direct evaluations of the posterior on a grid (we would just have to coerce those into the same format as the histogram that gets made from samples).

A simple call of the first function, `whist`, produces the following:

In [ ]:
h1 = cr.whist(test1D_1, plot=plt)

Note that, by default, this does some smoothing, specifically using a kernel density estimate method. This is why the blue curve (which is what is actually returned), is smoother than the raw histogram shown in the background. Note that the mode of the blue curve is not identical to the $x$ value where the the raw histogram is highest. This is certainly correct in this case, and it's often true that the mode of an unsmoothed histogram will be thrown off by monte carlo noise. In general, one should smooth until noisey features that you're pretty sure shouldn't be there are gone, and no more - assuming you're confident enough to make that call.

There may be times when you want to not smooth, or to have finer control over the smoothing (e.g., if you believe the identified mode is still off due to remaining noise) - see the docstring for more and/or play around yourself.


For reference, take a look at the returned object, a dictionary that straightforwardly holds the abscissae and estimated density of the PDF.

In [ ]:
h1

Next, assuming this looks reasonable, we feed the output of `whist` into `whist_ci`:

In [ ]:
ci1 = cr.whist_ci(h1, plot=plt)

This should look familiar from the notes. The plot shows the mode and the 1 and 2 sigma intervals, whose bounding probabilities are indicated by the horizontal lines. Here you should look at the intersection of the blue/solid and green/dashed lines with the black/solid PDF - if they aren't all coming together in the same place, then the underlying histogram needs to be binned more finely.

The return value contains a number of useful (and some redundant) values. Of most interest are the `mode` and interval `min` and `max` values. `low` and `high` are just the distance of `min` and `max` from the mode. Finally, `center` is the average of `min` and `max`, and `width` is half the difference between `min` and `max`. These are convenient for cases where the CI is symmetric about the mode (at the precision we would be reporting them).

In [ ]:
ci1

In text, we would normally report the constraint as $\mathrm{mode}^{+\mathrm{high}}_{-\mathrm{low}}$, or $\mathrm{mode}\pm\mathrm{width}$ if the CI is symmetric.

Next, compute the alternative "best fit" and CI's using the median/quantiles method from the notes. Store the median as a scalar and the intervals as shape (2,) arrays holding the endpoints of each interval.

In [ ]:
TBC()
# med1 = 
# quant1_1 = 
# quant1_2 = 

print(med1, quant1_1, quant1_2)

Even though their use is not recommended, we'll also compare to the mean/standard deviation.

In [ ]:
mea1 = np.mean(test1D_1)
std1 = np.std(test1D_1)
print(mea1, std1)

Here we'll show the median/quantile summary (green) and the mean/standard deviation summary (red) over the plot from earlier. You can see that none of them quite agree, and the mean/standard deviation $2\sigma$ interval includes negative values where the PDF is zero!

In [ ]:
ci1 = cr.whist_ci(h1, plot=plt, plot_levels=False)
plt.plot(quant1_2, [2.,2.], 'g')
plt.plot(quant1_1, [2.05,2.05], 'g')
plt.plot(med1, 2.05, 'go')
plt.plot(mea1, 2.55, 'ro')
plt.plot([mea1-std1, mea1+std1], [2.55,2.55], 'r')
plt.plot([mea1-2*std1, mea1+2*std1], [2.5,2.5], 'r')

Go through the same procedure for `test1D_2` next.

In [ ]:
h2 = cr.whist(test1D_2, plot=plt)

In [ ]:
TBC()
# med2 = 
# quant2_1 = 
# quant2_2 = 

print(med2, quant2_1, quant2_2)

In [ ]:
mea2 = np.mean(test1D_2)
std2 = np.std(test1D_2)
print(mea2, std2)

In [ ]:
ci2 = cr.whist_ci(h2, plot=plt, plot_levels=False)
plt.plot(quant2_2, [0.2,0.2], 'g')
plt.plot(quant2_1, [0.205,0.205], 'g')
plt.plot(med2, 0.205, 'go')
plt.plot(mea2, 0.155, 'ro')
plt.plot([mea2-std2, mea2+std2], [0.155,0.155], 'r')
plt.plot([mea2-2*std2, mea2+2*std2], [0.15,0.15], 'r')
ci2

Interesting... this is an example of a multimodal PDF where the $1\sigma$ highest-density CI is multiply connected (i.e. not contiguous). Note that `ci2` contains _two_ entries for `level` 0.683 to reflect this. The corresponding values in `prob` show the integrated probability in each of them, if that's useful.

## 2D regions

Next we have the case of 2D credible regions (CR's), where we'll take `test1D_1` and `test1D_2` to be samples of two parameters from the same model.

To find the HPD regions, we have analogous functions to the ones used above. However, `whist2d` does not have a KDE smoothing option, so in general you will want to manually adjust the number of bins (in each parameter) and the size of the smoothing kernel (in units of bins). For example, compare the following:

In [ ]:
h3 = cr.whist2d(test1D_1, test1D_2, bins=25, smooth=0, plot=True)

In [ ]:
h3 = cr.whist2d(test1D_1, test1D_2, bins=50, smooth=1, plot=True)

The return value of this function is a dictionary with `x`, `y` and `z` entries, corresponding to the two parameter values at the bin centers, and the density in each bin. These have dimensionality 1, 1, and 2, respectively.

To find the CR's, we call `whist2d_ci`.

In [ ]:
ci3 = cr.whist2d_ci(h3)

This returns the mode and the list of points making up the contours in the above plot in a slightly complicated structure that you can examine if you're interested.

One should test whether the amount of smoothing used is too much. (This is important in the 1D case also, but the KDE smoothing rarely has this issue.) Overlaying smoothed and less smoothed contours is a useful way of doing this.

In [ ]:
h3 = cr.whist2d(test1D_1, test1D_2, bins=100, smooth=0);
ci3 = cr.whist2d_ci(h3, mode_fmt=None, contour_color='r');
h3 = cr.whist2d(test1D_1, test1D_2, bins=50, smooth=1);
ci3 = cr.whist2d_ci(h3, mode_fmt=None);

In this comparison, the more smoothed contours should look like... a smoothed version of the squiggly ones, as opposed to being much broader.

Note that the function that extracts the contours in `whist2d_ci` (distinct from the one that _plots_ them) is not foolproof - it seems especially likely to screw up if there are too many complicated squiggles. So it is (a) extra-important to do this check, and (b) best to use `whist2d` for the check rather than the function (below) that simply plots contours once you've found them. Once you know what the contours are supposed to look like, you can make sure nothing pathalogical has happened when they get re-plotted (as below).

There is no 2D analog of the median/quantile convention for CI's, but we can compare with the equivalent (not recommended) regions defined via the parameter means and covariance.

In [ ]:
cr.ci2D_plot(ci3['contours'][1], plt) # this is how you plot contours from whist2d_ci without having to
cr.ci2D_plot(ci3['contours'][0], plt) # repeat all the computations

cv = np.cov(test1D_1, test1D_2)
cr.cov_ellipse(cv, center=[mea1,mea2], level=0.68268949, plot=plt, fmt='r--');
cr.cov_ellipse(cv, center=[mea1,mea2], level=0.95449974, plot=plt, fmt='r--');

Note that the stand-alone contour plotting function adds a single contour level at a time, so it was called twice.

Options to the various lower-level `matplotlib` functions that are called can be used to customize things, e.g.:

In [ ]:
cr.ci2D_plot(ci3['contours'][1], plt, fill=True, fill_kwargs={'color':str(0.954)})
cr.ci2D_plot(ci3['contours'][0], plt, fill=True, fill_kwargs={'color':str(0.683)})

An increasingly common way to visualize multiple 1D and 2D marginalized PDFs from a high-dimensional posterior distribution is the so-called "triangle plot" that has the 1D PDFs on its diagonal and each corresponding 2D PDF off the diagonal, in a triangular matrix of plots. For example, this call finds all of the CI's and CR's (note that it has to pop up a contour plot due to the way `whist2d_ci` works internally).

In [ ]:
tri = cr.whist_triangle(np.array([test1D_1, test1D_2]).T, bins=50, smooth2D=1)

We can display the triangle thusly:

In [ ]:
cr.whist_triangle_plot(tri, paramNames=[r'$x$', r'$y$']);

Again, lots of options can be customized, and other things can be added to any of the panels via the returned array of `matplotlib` axes.

In [ ]:
f, a = cr.whist_triangle_plot(tri, paramNames=[r'$x$', r'$y$'], linecolor1D='b',
                              fillcolors2D=[(0.026, 0.818, 1.000), (0.714, 0.936, 1.000)]);
cr.cov_ellipse(cv, center=[mea1,mea2], level=0.68268949, plot=a[1][0], fmt='r--');
cr.cov_ellipse(cv, center=[mea1,mea2], level=0.95449974, plot=a[1][0], fmt='r--');

Note that there are other packages that can be better for getting a quick look at plots like this, in particular `corner` and `pygtc`. We'll use both of these in later tutorials. They have the advantage that you get the whole triangle plot from a single function call. The downside is that all the calculations are redone each time you call the function, and it isn't so straightforward to do the sanity checks mentioned above. So we recommend using one of those packages only for quickly visualizing samples from a PDF. In contrast, with `incredible` it's straightforward to do the hard calculations once, and then tweak the display to your heart's content, so it might be more suitable for publication graphics (in addition to providing the CI calculations). To summarize the options as of this writing:
* `corner`: probably the quickest way to create a triangle plot showing the key information (note that the default 2D probability levels are not the conventional ones)
* `pygtc`: looks nicer than `corner`; can put multiple posteriors on the same plot
* `incredible`: provides fine-tuned control over CR calculations, and ability to save results rather than just a plot, but needs some hand holding